<a href="https://colab.research.google.com/github/JScottLopez/JScottLopez/blob/main/Copy_of_Lopez_J_DDS_8555v1_Week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DDS-8555v1: Week 1
Joseph Lopez
Feb 21, 2025
Regression and Classifier Metrics

In [ ]:
#Install the necessary packages
!pip install nbconvert
from sklearn import datasets
import pandas as pd
import numpy as np

In [ ]:
#Load the iris dataset in Python
iris= pd.DataFrame(datasets.load_iris().data)

iris.columns = datasets.load_iris().feature_names

iris['type'] = datasets.load_iris().target

iris['type']=iris['type'].astype('object')

iris

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),type
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [ ]:
#Create a new feature by multiplying sepal length by sepal width and dividing by petal length * petal width.
iris['new']=(iris.iloc[:,0]*iris.iloc[:,1])/(iris.iloc[:,2]*iris.iloc[:,3])

print(iris.head())

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

  type        new  
0    0  63.750000  
1    0  52.500000  
2    0  57.846154  
3    0  47.533333  
4    0  64.285714  


In [ ]:
#Sample 80% of the data for a training set stratifying on ‘type’
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test = tts(iris.iloc[:,0:4], iris.iloc[:,4], test_size=0.2, random_state=42,
stratify=iris.iloc[:,4])

In [ ]:
#Mean of petal length calculated only on the training data
#Mean of sepal length minus petal width calculated only on the training data
est1=np.mean(X_train['petal length (cm)'])
est2=np.mean(X_train['sepal length (cm)']-X_train['petal width (cm)'])

y_pred1 = np.full(len(X_test), est1)
y_pred2 = np.full(len(X_test), est2)

def calculate_metrics(y_true, y_pred):
    me = np.mean(y_true - y_pred)
    mpe = np.mean((y_true - y_pred) / y_true) * 100
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    mae = np.mean(np.abs(y_true - y_pred))
    mse = np.mean((y_true - y_pred)**2)
    return me, mpe, mape, mae, mse

#Calculate metrics
metrics1 = calculate_metrics(X_test['sepal width (cm)'], y_pred1)
metrics2 = calculate_metrics(X_test['sepal width (cm)'], y_pred2)

#Print results
print("Estimator 1 - mean of petal length metrics:")
print(f"ME: {metrics1[0]:.4f}")
print(f"MPE: {metrics1[1]:.4f}%")
print(f"MAPE: {metrics1[2]:.4f}%")
print(f"MAE: {metrics1[3]:.4f}")
print(f"MSE: {metrics1[4]:.4f}")
print("\nEstimator 2 - mean of sepal length minus petal width metrics:")
print(f"ME: {metrics2[0]:.4f}")
print(f"MPE: {metrics2[1]:.4f}%")
print(f"MAPE: {metrics2[2]:.4f}%")
print(f"MAE: {metrics2[3]:.4f}")
print(f"MSE: {metrics2[4]:.4f}")

Estimator 1 - mean of petal length metrics:
ME: -0.6767
MPE: -23.7499%
MAPE: 24.1859%
MAE: 0.6940
MSE: 0.6018

Estimator 2 - mean of sepal length minus petal width metrics:
ME: -1.5433
MPE: -52.1981%
MAPE: 52.1981%
MAE: 1.5433
MSE: 2.5258


In [ ]:
#Up to 1st quantile of sepal length = type 0, >1st up to 2d quantile = type 1, >2d quantile = type 2
#Up to 2d quantile of sepal length = type 0, >2d up to 3d quantile = type 1, >3d quantile = type 2
from numpy import percentile
from sklearn.metrics import classification_report as cr

est3=percentile(X_train['sepal length (cm)'], [25, 50])
y_hat=np.zeros(len(y_test))
y_hat[X_test['sepal length (cm)']>est3[0]]=1
y_hat[X_test['sepal length (cm)']>est3[1]]=2
y_hat=y_hat.astype('int')
print("Estimator 3 - sepal length 2d quantile metrics:")
print(cr(y_test.astype('int'),y_hat))

est4=percentile(X_train['sepal length (cm)'], [50,75])
y_hat2=np.zeros(len(y_test))
y_hat2[X_test['sepal length (cm)']>est4[0]]=1
y_hat2[X_test['sepal length (cm)']>est4[1]]=2
y_hat2=y_hat2.astype('int')
print("Estimator 4 - sepal length 3d quantile metrics:")
print(cr(y_test.astype('int'),y_hat2))

Estimator 3 - sepal length 2d quantile metrics:
              precision    recall  f1-score   support

           0       0.71      0.50      0.59        10
           1       0.33      0.20      0.25        10
           2       0.59      1.00      0.74        10

    accuracy                           0.57        30
   macro avg       0.55      0.57      0.53        30
weighted avg       0.55      0.57      0.53        30

Estimator 4 - sepal length 3d quantile metrics:
              precision    recall  f1-score   support

           0       0.69      0.90      0.78        10
           1       0.38      0.30      0.33        10
           2       0.67      0.60      0.63        10

    accuracy                           0.60        30
   macro avg       0.58      0.60      0.58        30
weighted avg       0.58      0.60      0.58        30

